### RAG Adaptativo (Adaptive RAG)

**¿Qué es Adaptive RAG?**

Adaptive RAG es una técnica avanzada que combina lo mejor de múltiples patrones RAG mediante **enrutamiento inteligente** y **auto-corrección**. El sistema toma decisiones adaptativas en cada paso:

1. **Enrutamiento inicial**: Decide si buscar en vectorstore local o en la web
2. **Evaluación de relevancia**: Califica documentos recuperados
3. **Detección de alucinaciones**: Verifica que la respuesta esté fundamentada en los documentos
4. **Validación de respuesta**: Confirma que la respuesta realmente conteste la pregunta
5. **Auto-corrección**: Si algo falla, reescribe la consulta y reintenta

Este es el enfoque más completo y robusto, ideal para aplicaciones de producción donde la calidad es crítica.

In [ ]:
# Importar librería para manejo de variables de entorno
import os
# Importar función para cargar variables desde archivo .env
from dotenv import load_dotenv

# Cargar todas las variables de entorno desde el archivo .env
# Esto permite mantener las claves API de forma segura fuera del código
load_dotenv()

# Configurar la clave API de OpenAI (para embeddings, LLM y evaluadores)
# Necesaria para usar GPT-4o-mini y text-embedding-ada-002
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Configurar la clave API de Tavily (motor de búsqueda web para IA)
# Se usa cuando el router decide que la consulta necesita búsqueda web
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

In [ ]:
### Construir Índice Vectorial

# Importar divisor de texto que respeta los tokens del modelo
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Importar cargador de documentos desde páginas web
from langchain_community.document_loaders import WebBaseLoader
# Importar FAISS, base de datos vectorial en memoria (rápida y eficiente)
from langchain_community.vectorstores import FAISS
# Importar embeddings de OpenAI para convertir texto en vectores
from langchain_openai import OpenAIEmbeddings

### from langchain_cohere import CohereEmbeddings  # Alternativa: embeddings de Cohere

# Configurar el modelo de embeddings
# OpenAIEmbeddings() usa por defecto text-embedding-ada-002
embd = OpenAIEmbeddings()

# Definir las URLs de los documentos a indexar
# Artículos del blog de Lilian Weng sobre IA, agentes y LLMs
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",           # Agentes de IA
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",  # Ingeniería de prompts
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",      # Ataques adversarios a LLMs
]

# Cargar los documentos desde las URLs
# WebBaseLoader carga el contenido HTML de cada URL
docs = [WebBaseLoader(url).load() for url in urls]

# Aplanar la lista de listas en una sola lista de documentos
# Convierte [[doc1], [doc2], [doc3]] en [doc1, doc2, doc3]
docs_list = [item for sublist in docs for item in sublist]

# Crear un divisor de texto basado en tokens (tiktoken de OpenAI)
# chunk_size=500: cada fragmento tendrá máximo 500 tokens
# chunk_overlap=50: habrá un solapamiento de 50 tokens entre fragmentos consecutivos
# El overlap ayuda a mantener contexto entre chunks y mejora la recuperación
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=50
)

# Dividir los documentos en fragmentos más pequeños
# Fragmentos pequeños = recuperación más precisa
doc_splits = text_splitter.split_documents(docs_list)

# Crear el vectorstore (base de datos vectorial) con FAISS
# from_documents() convierte cada fragmento en vector y los almacena
vectorstore = FAISS.from_documents(
    documents=doc_splits,
    embedding=OpenAIEmbeddings()
)

# Crear un retriever (recuperador) desde el vectorstore
# Interfaz que permite buscar documentos similares a una consulta
# Por defecto usa similitud coseno y retorna k=4 documentos
retriever = vectorstore.as_retriever()

In [ ]:
### Enrutador (Router)

# Importar Literal para definir tipos con valores específicos
from typing import Literal
# Importar componentes para crear prompts estructurados
from langchain_core.prompts import ChatPromptTemplate
# Importar el modelo de chat de OpenAI
from langchain_openai import ChatOpenAI
# Importar Pydantic para validación de datos
from pydantic import BaseModel, Field

# Definir un modelo de datos con Pydantic para el enrutamiento
# Este modelo fuerza al LLM a responder con una de las dos opciones
class RouteQuery(BaseModel):
    """Enruta una consulta del usuario a la fuente de datos más relevante."""

    # Campo que contendrá la decisión de enrutamiento
    # Literal["vectorstore", "web_search"] limita las opciones a solo estas dos
    # ... indica que el campo es requerido
    datasource: Literal["vectorstore", "web_search"] = Field(
        ...,
        description="Dada una pregunta del usuario, elige enrutarla a búsqueda web o a un vectorstore.",
    )

# Crear una instancia del LLM para enrutamiento
# model="gpt-4o-mini": modelo rápido y económico (reemplazo de gpt-3.5-turbo)
# temperature=0: respuestas determinísticas (sin aleatoriedad)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Crear un LLM con salida estructurada según el modelo RouteQuery
# with_structured_output() hace que el LLM responda exactamente en el formato esperado
structured_llm_router = llm.with_structured_output(RouteQuery)

# Definir el prompt del sistema que instruye al LLM sobre su tarea de enrutamiento
# Este prompt es crítico: define los criterios para decidir entre vectorstore y web
system = """Eres un experto en enrutar una pregunta del usuario a un vectorstore o búsqueda web.
El vectorstore contiene documentos relacionados con agentes, ingeniería de prompts y ataques adversarios.
Usa el vectorstore para preguntas sobre estos temas. Para todo lo demás, usa búsqueda web."""

# Crear la plantilla de prompt completa
# from_messages() crea un chat con dos roles: system (instrucciones) y human (input)
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),  # Instrucciones del sistema
        ("human", "{question}"),  # Template para la pregunta del usuario
    ]
)

# Encadenar el prompt con el LLM estructurado usando el operador |
# Esto crea un "chain": prompt → LLM → salida estructurada (RouteQuery)
question_router = route_prompt | structured_llm_router

# Ejemplo 1: Prueba con pregunta que requiere búsqueda web
# "Cricket world cup 2023" no está en los documentos locales (agentes, prompts, ataques)
# El router debe decidir: web_search
print(
    question_router.invoke(
        {"question": "Who won the Cricket world cup 2023 "}
    )
)
# Output esperado: datasource='web_search'

In [ ]:
# Ejemplo 2: Prueba con pregunta que debería ir al vectorstore local
# "What are the types of agent memory?" está relacionada con agentes (tema en el vectorstore)
# El router debe decidir: vectorstore
print(question_router.invoke({"question": "What are the types of agent memory?"}))
# Output esperado: datasource='vectorstore'

In [ ]:
### Evaluador de Recuperación (Retrieval Grader)

# Definir un modelo de datos con Pydantic para evaluar relevancia de documentos
class GradeDocuments(BaseModel):
    """Puntuación binaria para verificar la relevancia de documentos recuperados."""

    # Campo que contendrá la evaluación: 'yes' si es relevante, 'no' si no lo es
    binary_score: str = Field(
        description="Los documentos son relevantes a la pregunta, 'yes' o 'no'"
    )

# Crear una instancia del LLM para evaluación de documentos
# Usamos gpt-4o-mini para mantener costos bajos (muchas evaluaciones)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Crear un LLM con salida estructurada según el modelo GradeDocuments
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Definir el prompt del sistema para evaluar relevancia de documentos
# IMPORTANTE: La prueba NO debe ser estricta, el objetivo es filtrar recuperaciones ERRÓNEAS
# Es mejor dejar pasar un documento mediocre que filtrar uno potencialmente útil
system = """Eres un evaluador que analiza la relevancia de un documento recuperado respecto a una pregunta del usuario. \n 
    Si el documento contiene palabra(s) clave o significado semántico relacionado con la pregunta del usuario, califícalo como relevante. \n
    No necesita ser una prueba estricta. El objetivo es filtrar recuperaciones erróneas. \n
    Da una puntuación binaria 'yes' o 'no' para indicar si el documento es relevante a la pregunta."""

# Crear la plantilla de prompt para evaluación
# Recibe el documento recuperado y la pregunta del usuario
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),  # Instrucciones del sistema
        ("human", "Documento recuperado: \n\n {document} \n\n Pregunta del usuario: {question}"),
    ]
)

# Encadenar el prompt con el LLM estructurado
# Flujo: prompt (con document y question) → LLM → GradeDocuments
retrieval_grader = grade_prompt | structured_llm_grader

# Ejemplo de uso: evaluar si un documento es relevante
question = "agent memory"  # Pregunta sobre memoria de agentes

# Recuperar documentos similares usando el retriever
docs = retriever.invoke(question)

# Obtener el contenido del segundo documento (índice 1)
doc_txt = docs[1].page_content

# Evaluar la relevancia del documento
# El LLM analiza si el documento contiene información sobre memoria de agentes
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))
# Output esperado: binary_score='yes' (el documento habla sobre memoria de agentes)

In [ ]:
### Generación RAG

# Importar hub de LangChain para acceder a prompts compartidos
from langchain import hub
# Importar parser para convertir la salida del LLM en string
from langchain_core.output_parsers import StrOutputParser

# Obtener un prompt predefinido del hub de LangChain
# "rlm/rag-prompt" es un prompt optimizado para RAG
prompt = hub.pull("rlm/rag-prompt")

# Crear una instancia del LLM para generación de respuestas
# model_name="gpt-4o-mini": modelo rápido y económico
# temperature=0: respuestas determinísticas
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Función auxiliar para formatear documentos
# Convierte una lista de objetos Document en un string con saltos de línea
def format_docs(docs):
    # Une el contenido de cada documento con doble salto de línea
    return "\n\n".join(doc.page_content for doc in docs)

# Crear la cadena RAG completa usando el operador pipe (|)
# Flujo: prompt → LLM → parser de string
rag_chain = prompt | llm | StrOutputParser()

# Ejecutar la cadena RAG con los documentos recuperados y la pregunta
# invoke() genera una respuesta basada en el contexto de los documentos
generation = rag_chain.invoke({"context": docs, "question": question})

# Imprimir la respuesta generada
# Esta respuesta estará basada en el contenido de los documentos recuperados
print(generation)
# Output: Una respuesta sobre los tipos de memoria de agentes (short-term y long-term)

In [ ]:
### Evaluador de Alucinaciones (Hallucination Grader)

# Definir un modelo de datos para detectar alucinaciones
# Una "alucinación" ocurre cuando el LLM genera información NO presente en los documentos
class GradeHallucinations(BaseModel):
    """Puntuación binaria para detectar alucinaciones en la respuesta generada."""

    # Campo que indica si la respuesta está fundamentada en los hechos
    binary_score: str = Field(
        description="La respuesta está fundamentada en los hechos, 'yes' o 'no'"
    )

# Crear una instancia del LLM para detectar alucinaciones
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Crear un LLM con salida estructurada
structured_llm_grader = llm.with_structured_output(GradeHallucinations)

# Definir el prompt del sistema para detección de alucinaciones
# El evaluador verifica si la generación del LLM está SOPORTADA por los documentos recuperados
system = """Eres un evaluador que analiza si una generación de LLM está fundamentada en / soportada por un conjunto de hechos recuperados. \n 
     Da una puntuación binaria 'yes' o 'no'. 'Yes' significa que la respuesta está fundamentada en / soportada por el conjunto de hechos."""

# Crear la plantilla de prompt
# Recibe: documents (hechos recuperados) y generation (respuesta del LLM)
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Conjunto de hechos: \n\n {documents} \n\n Generación del LLM: {generation}"),
    ]
)

# Encadenar el prompt con el LLM estructurado
# Flujo: prompt (con documents y generation) → LLM → GradeHallucinations
hallucination_grader = hallucination_prompt | structured_llm_grader

# Evaluar si la generación contiene alucinaciones
# El evaluador compara la respuesta generada con los documentos originales
hallucination_grader.invoke({"documents": docs, "generation": generation})
# Output esperado: GradeHallucinations(binary_score='yes')
# Significa que la respuesta está fundamentada en los documentos

In [ ]:
### Evaluador de Respuestas (Answer Grader)

# Definir un modelo de datos para evaluar si la respuesta contesta la pregunta
# Este evaluador verifica si la respuesta REALMENTE responde lo que el usuario preguntó
class GradeAnswer(BaseModel):
    """Puntuación binaria para evaluar si la respuesta aborda la pregunta."""

    # Campo que indica si la respuesta aborda/resuelve la pregunta
    binary_score: str = Field(
        description="La respuesta aborda la pregunta, 'yes' o 'no'"
    )

# Crear una instancia del LLM para evaluar respuestas
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Crear un LLM con salida estructurada
structured_llm_grader = llm.with_structured_output(GradeAnswer)

# Definir el prompt del sistema para evaluación de respuestas
# El evaluador verifica si la respuesta RESUELVE/ABORDA la pregunta del usuario
system = """Eres un evaluador que analiza si una respuesta aborda / resuelve una pregunta \n 
     Da una puntuación binaria 'yes' o 'no'. 'Yes' significa que la respuesta resuelve la pregunta."""

# Crear la plantilla de prompt
# Recibe: question (pregunta del usuario) y generation (respuesta del LLM)
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Pregunta del usuario: \n\n {question} \n\n Generación del LLM: {generation}"),
    ]
)

# Encadenar el prompt con el LLM estructurado
# Flujo: prompt (con question y generation) → LLM → GradeAnswer
answer_grader = answer_prompt | structured_llm_grader

# Evaluar si la respuesta aborda la pregunta
# El evaluador compara la pregunta original con la respuesta generada
answer_grader.invoke({"question": question, "generation": generation})
# Output esperado: GradeAnswer(binary_score='yes')
# Significa que la respuesta sí contesta la pregunta sobre memoria de agentes

In [ ]:
### Reescritor de Preguntas (Question Re-writer)

# Crear una instancia del LLM para reescribir preguntas
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Definir el prompt del sistema para reescritura de preguntas
# Objetivo: optimizar la pregunta para RECUPERACIÓN EN VECTORSTORE
# Diferencia vs CRAG: aquí optimizamos para vectorstore, CRAG optimiza para web
system = """Eres un reescritor de preguntas que convierte una pregunta de entrada en una mejor versión optimizada \n 
     para recuperación en vectorstore. Observa la entrada e intenta razonar sobre la intención semántica / significado subyacente."""

# Crear la plantilla de prompt para reescritura
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Aquí está la pregunta inicial: \n\n {question} \n Formula una pregunta mejorada.",
        ),
    ]
)

# Crear la cadena de reescritura
# Flujo: prompt con {question} → LLM → parser que convierte a string
question_rewriter = re_write_prompt | llm | StrOutputParser()

# Probar el reescritor con la pregunta de ejemplo
# La pregunta "agent memory" es demasiado vaga
# El reescritor debe generar una versión más detallada y específica
question_rewriter.invoke({"question": question})
# Output esperado: Una pregunta más específica como:
# "What are the key concepts and techniques related to agent memory in artificial intelligence?"

In [ ]:
### Herramienta de Búsqueda Web

# Importar la herramienta de búsqueda de Tavily
# Tavily es un motor de búsqueda optimizado para agentes de IA
from langchain_community.tools.tavily_search import TavilySearchResults

# Crear una instancia de la herramienta de búsqueda web
# k=3: devuelve los 3 mejores resultados de búsqueda
# Se usa cuando el router decide que la consulta necesita información de la web
web_search_tool = TavilySearchResults(k=3)

In [ ]:
### Definición del Estado del Grafo

# Importar List para tipar listas
from typing import List
# Importar TypedDict para crear diccionarios con tipos específicos
from typing_extensions import TypedDict

# Definir la clase GraphState que representa el estado del grafo en LangGraph
# TypedDict permite especificar los tipos de datos de cada clave del diccionario
# Este estado se pasa entre todos los nodos del grafo
class GraphState(TypedDict):
    """
    Representa el estado de nuestro grafo.

    Atributos:
        question: La pregunta del usuario (original o reescrita)
        generation: La respuesta generada por el LLM
        documents: Lista de documentos recuperados (de vectorstore o web)
    """

    # Pregunta del usuario (puede ser transformada por el reescritor)
    question: str
    
    # Respuesta final generada por el LLM usando RAG
    generation: str
    
    # Lista de documentos que se usarán como contexto
    # Puede contener documentos del vectorstore local o resultados de búsqueda web
    documents: List[str]

In [ ]:
### Funciones de los Nodos del Grafo y Aristas Condicionales

# Importar Document para crear objetos de documento
from langchain.schema import Document


# ========== NODOS DEL GRAFO ==========

# NODO 1: Recuperar documentos del vectorstore local
def retrieve(state):
    """
    Recupera documentos relevantes desde el vectorstore local.
    
    Args:
        state (dict): El estado actual del grafo
    
    Returns:
        state (dict): Estado actualizado con los documentos recuperados
    """
    print("---RECUPERAR---")
    # Extraer la pregunta del estado
    question = state["question"]

    # Recuperar documentos similares usando el retriever
    # Busca en el vectorstore FAISS usando similitud coseno
    documents = retriever.invoke(question)
    
    # Retornar el estado actualizado
    return {"documents": documents, "question": question}


# NODO 2: Generar respuesta usando RAG
def generate(state):
    """
    Genera una respuesta usando los documentos como contexto.
    
    Args:
        state (dict): El estado actual del grafo
    
    Returns:
        state (dict): Estado actualizado con la generación del LLM
    """
    print("---GENERAR---")
    # Extraer pregunta y documentos del estado
    question = state["question"]
    documents = state["documents"]

    # Generar respuesta usando la cadena RAG
    # rag_chain toma el contexto y la pregunta, y genera una respuesta
    generation = rag_chain.invoke({"context": documents, "question": question})
    
    # Retornar el estado completo con la respuesta generada
    return {"documents": documents, "question": question, "generation": generation}


# NODO 3: Evaluar relevancia de documentos
def grade_documents(state):
    """
    Determina si los documentos recuperados son relevantes a la pregunta.
    Filtra documentos no relevantes.
    
    Args:
        state (dict): El estado actual del grafo
    
    Returns:
        state (dict): Estado con documentos filtrados (solo relevantes)
    """

    print("---VERIFICAR RELEVANCIA DE DOCUMENTOS A LA PREGUNTA---")
    # Extraer pregunta y documentos del estado
    question = state["question"]
    documents = state["documents"]

    # Lista para almacenar solo los documentos relevantes
    filtered_docs = []
    
    # Evaluar cada documento individualmente
    for d in documents:
        # Usar el evaluador (retrieval_grader) para calificar el documento
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        # Extraer la puntuación binaria (yes/no)
        grade = score.binary_score
        
        # Si el documento es relevante, agregarlo a la lista filtrada
        if grade == "yes":
            print("---CALIFICACIÓN: DOCUMENTO RELEVANTE---")
            filtered_docs.append(d)
        # Si no es relevante, saltarlo
        else:
            print("---CALIFICACIÓN: DOCUMENTO NO RELEVANTE---")
            continue
    
    # Retornar estado con solo los documentos relevantes
    return {"documents": filtered_docs, "question": question}


# NODO 4: Transformar/reescribir la pregunta
def transform_query(state):
    """
    Transforma la pregunta para producir una versión mejorada.
    Se usa cuando los documentos no son relevantes o la respuesta no es útil.
    
    Args:
        state (dict): El estado actual del grafo
    
    Returns:
        state (dict): Estado con la pregunta reescrita
    """

    print("---TRANSFORMAR CONSULTA---")
    # Extraer pregunta y documentos del estado
    question = state["question"]
    documents = state["documents"]

    # Reescribir la pregunta usando el reescritor (question_rewriter)
    # Esto optimiza la pregunta para mejor recuperación en vectorstore
    better_question = question_rewriter.invoke({"question": question})
    
    # Retornar estado con la pregunta mejorada
    return {"documents": documents, "question": better_question}


# NODO 5: Realizar búsqueda web
def web_search(state):
    """
    Realiza búsqueda web usando Tavily.
    Se usa cuando el router decide que la consulta necesita información de la web.
    
    Args:
        state (dict): El estado actual del grafo
    
    Returns:
        state (dict): Estado con documentos de búsqueda web
    """

    print("---BÚSQUEDA WEB---")
    # Extraer pregunta del estado
    question = state["question"]

    # Realizar búsqueda web usando la herramienta Tavily
    # invoke() envía la consulta a Tavily y obtiene k=3 resultados
    docs = web_search_tool.invoke({"query": question})
    
    # Unir el contenido de todos los resultados web en un solo string
    web_results = "\n".join([d["content"] for d in docs])
    
    # Crear un objeto Document con los resultados web
    # IMPORTANTE: Reemplaza documents (no append), porque viene del router directo a web
    web_results = Document(page_content=web_results)

    # Retornar estado con los resultados web como documentos
    return {"documents": web_results, "question": question}


# ========== ARISTAS CONDICIONALES (DECISIONES) ==========

# DECISIÓN 1: ¿Enrutar a vectorstore o búsqueda web?
def route_question(state):
    """
    Enruta la pregunta a búsqueda web o RAG (vectorstore).
    Esta es la PRIMERA decisión del flujo Adaptive RAG.
    
    Args:
        state (dict): El estado actual del grafo
    
    Returns:
        str: Nombre del siguiente nodo ("web_search" o "vectorstore")
    """

    print("---ENRUTAR PREGUNTA---")
    # Extraer la pregunta del estado
    question = state["question"]
    
    # Usar el router para decidir la fuente de datos
    # El router analiza la pregunta y decide: ¿vectorstore o web?
    source = question_router.invoke({"question": question})
    
    # Si el router decide búsqueda web
    if source.datasource == "web_search":
        print("---ENRUTAR PREGUNTA A BÚSQUEDA WEB---")
        # Ir directamente al nodo web_search
        return "web_search"
    # Si el router decide vectorstore
    elif source.datasource == "vectorstore":
        print("---ENRUTAR PREGUNTA A RAG---")
        # Ir al nodo retrieve (recuperar de vectorstore)
        return "vectorstore"


# DECISIÓN 2: ¿Generar respuesta o transformar consulta?
def decide_to_generate(state):
    """
    Determina si generar una respuesta o reescribir la pregunta.
    Se ejecuta DESPUÉS de evaluar la relevancia de los documentos.
    
    Args:
        state (dict): El estado actual del grafo
    
    Returns:
        str: Nombre del siguiente nodo ("transform_query" o "generate")
    """

    print("---EVALUAR DOCUMENTOS CALIFICADOS---")
    # Extraer documentos filtrados del estado
    state["question"]
    filtered_documents = state["documents"]

    # Si NO hay documentos relevantes (todos fueron filtrados)
    if not filtered_documents:
        # Todos los documentos fueron filtrados por irrelevantes
        # Necesitamos reescribir la pregunta y reintentar
        print(
            "---DECISIÓN: TODOS LOS DOCUMENTOS NO SON RELEVANTES A LA PREGUNTA, TRANSFORMAR CONSULTA---"
        )
        # Ir al nodo transform_query
        return "transform_query"
    # Si SÍ hay documentos relevantes
    else:
        # Tenemos al menos un documento relevante, podemos generar respuesta
        print("---DECISIÓN: GENERAR---")
        # Ir al nodo generate
        return "generate"


# DECISIÓN 3: ¿La generación es útil y está fundamentada?
def grade_generation_v_documents_and_question(state):
    """
    Determina si la generación está fundamentada en los documentos y responde la pregunta.
    Esta es la evaluación FINAL que valida la calidad de la respuesta.
    
    Args:
        state (dict): El estado actual del grafo
    
    Returns:
        str: Decisión del siguiente paso ("useful", "not useful", "not supported")
    """

    print("---VERIFICAR ALUCINACIONES---")
    # Extraer todos los valores necesarios del estado
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    # PASO 1: Verificar si hay alucinaciones
    # ¿La respuesta está fundamentada en los documentos?
    score = hallucination_grader.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = score.binary_score

    # Si la respuesta SÍ está fundamentada en los documentos (no hay alucinaciones)
    if grade == "yes":
        print("---DECISIÓN: GENERACIÓN ESTÁ FUNDAMENTADA EN DOCUMENTOS---")
        
        # PASO 2: Verificar si la respuesta contesta la pregunta
        print("---CALIFICAR GENERACIÓN vs PREGUNTA---")
        score = answer_grader.invoke({"question": question, "generation": generation})
        grade = score.binary_score
        
        # Si la respuesta SÍ contesta la pregunta
        if grade == "yes":
            print("---DECISIÓN: GENERACIÓN ABORDA LA PREGUNTA---")
            # ¡Éxito! La respuesta es útil y fundamentada
            return "useful"
        # Si la respuesta NO contesta la pregunta
        else:
            print("---DECISIÓN: GENERACIÓN NO ABORDA LA PREGUNTA---")
            # La respuesta está fundamentada pero no contesta lo que se preguntó
            # Necesitamos reescribir la pregunta y reintentar
            return "not useful"
    # Si la respuesta NO está fundamentada (contiene alucinaciones)
    else:
        print("---DECISIÓN: GENERACIÓN NO ESTÁ FUNDAMENTADA EN DOCUMENTOS, REINTENTAR---")
        # La respuesta tiene alucinaciones, regenerar
        return "not supported"

In [ ]:
### Construcción del Flujo de Trabajo con LangGraph

# Importar componentes de LangGraph para crear el flujo de trabajo
from langgraph.graph import END, StateGraph, START

# Crear una instancia del grafo con el estado definido (GraphState)
workflow = StateGraph(GraphState)

# Definir los nodos del grafo
# Cada nodo es una función que recibe y actualiza el estado
workflow.add_node("web_search", web_search)          # Nodo: Búsqueda web con Tavily
workflow.add_node("retrieve", retrieve)              # Nodo: Recuperar documentos del vectorstore
workflow.add_node("grade_documents", grade_documents)  # Nodo: Evaluar relevancia de documentos
workflow.add_node("generate", generate)              # Nodo: Generar respuesta final
workflow.add_node("transform_query", transform_query)  # Nodo: Reescribir pregunta

# ========== CONSTRUIR EL GRAFO ==========

# ARISTA CONDICIONAL 1: START → ¿web_search o retrieve?
# Esta es la PRIMERA decisión: el router decide la fuente de datos
workflow.add_conditional_edges(
    START,                    # Nodo origen: inicio del grafo
    route_question,           # Función de decisión: router
    {
        "web_search": "web_search",      # Si router dice "web" → ir a web_search
        "vectorstore": "retrieve",       # Si router dice "vectorstore" → ir a retrieve
    },
)

# ARISTA 2: web_search → generate
# Después de búsqueda web, generar respuesta directamente
# No necesita evaluación de relevancia (asumimos que Tavily da resultados relevantes)
workflow.add_edge("web_search", "generate")

# ARISTA 3: retrieve → grade_documents
# Después de recuperar documentos locales, evaluar su relevancia
workflow.add_edge("retrieve", "grade_documents")

# ARISTA CONDICIONAL 2: grade_documents → ¿transform_query o generate?
# Decisión basada en si hay documentos relevantes
workflow.add_conditional_edges(
    "grade_documents",        # Nodo origen
    decide_to_generate,       # Función de decisión
    {
        "transform_query": "transform_query",  # Si no hay docs relevantes → reescribir
        "generate": "generate",                # Si hay docs relevantes → generar
    },
)

# ARISTA 4: transform_query → retrieve
# Después de reescribir la pregunta, reintentar recuperación
# Esto crea un CICLO que permite múltiples intentos de recuperación
workflow.add_edge("transform_query", "retrieve")

# ARISTA CONDICIONAL 3: generate → ¿END, transform_query o generate?
# Esta es la evaluación FINAL de la calidad de la respuesta
workflow.add_conditional_edges(
    "generate",               # Nodo origen
    grade_generation_v_documents_and_question,  # Función de decisión compleja
    {
        "not supported": "generate",       # Si hay alucinaciones → regenerar
        "useful": END,                     # Si es útil y fundamentada → terminar
        "not useful": "transform_query",   # Si no contesta → reescribir y reintentar
    },
)

# Compilar el grafo en una aplicación ejecutable
app = workflow.compile()

# ========== FLUJO COMPLETO DEL ADAPTIVE RAG ==========
#
# START
#   ↓
# [route_question] ← DECISIÓN 1: ¿Vectorstore o Web?
#   ↓
#   ├─→ [web_search] → [generate] → [grade_generation...] ← DECISIÓN 3
#   │
#   └─→ [retrieve] → [grade_documents] ← DECISIÓN 2
#         ↓
#         ├─→ [generate] → [grade_generation...] ← DECISIÓN 3
#         │                   ↓
#         │                   ├─→ [useful] → END ✓
#         │                   ├─→ [not useful] → [transform_query] → [retrieve] (ciclo)
#         │                   └─→ [not supported] → [generate] (ciclo)
#         │
#         └─→ [transform_query] → [retrieve] (ciclo)
#
# CARACTERÍSTICAS CLAVE:
# - Enrutamiento inteligente inicial (web vs vectorstore)
# - Evaluación de relevancia de documentos
# - Detección de alucinaciones
# - Validación de que la respuesta conteste la pregunta
# - Auto-corrección mediante ciclos de retroalimentación
# - Múltiples intentos hasta obtener una respuesta de calidad

In [ ]:
# Ejecutar el flujo completo de Adaptive RAG
# Ejemplo 1: Pregunta que necesita búsqueda web
# "What is machine learning" no está en los documentos locales (agentes, prompts, ataques)
# El router debe enviarla a búsqueda web

app.invoke({"question": "What is machine learning"})

# FLUJO ESPERADO:
# 1. route_question decide: "web_search"
# 2. web_search busca en Tavily
# 3. generate crea respuesta con resultados web
# 4. grade_generation verifica: ¿fundamentada? ¿útil?
# 5. Si todo está bien: retorna respuesta final

In [ ]:
# Ejemplo 2: Pregunta que usa el vectorstore local
# "What is agent memory" está relacionada con agentes (tema en el vectorstore)
# El router debe enviarla al vectorstore local

app.invoke({"question": "What is agent memory"})

# FLUJO ESPERADO:
# 1. route_question decide: "vectorstore"
# 2. retrieve busca en FAISS
# 3. grade_documents evalúa relevancia de cada documento
# 4. Si hay docs relevantes: generate crea respuesta
# 5. grade_generation verifica calidad
# 6. Si no hay docs relevantes: transform_query → retrieve (reintenta)
# 7. Si la respuesta tiene alucinaciones: generate (regenera)
# 8. Si la respuesta no contesta: transform_query → retrieve (reintenta)
# 9. Si todo está bien: retorna respuesta final